In [43]:
import requests
import pandas as pd
import numpy as np
import json
import datetime
from datetime import date, timedelta

In [24]:
s_date = date(2016, 4, 1)
delta=366
responses=[]
start_date=s_date
for i in range(5):
    end_date=start_date + timedelta(days = delta)
    print(start_date)
    print(end_date)
    r=requests.get('https://docs.openaq.org/v2/measurements?date_from='+str(start_date)+'T00%3A00%3A00%2B00%3A00&date_to='+str(end_date)+'T03%3A55%3A00%2B00%3A00&limit=100000&page=1&offset=0&sort=desc&radius=1000&country_id=GB&location_id=155&order_by=datetime')
    responses += [r.json()]
    start_date=end_date

2016-04-01
2017-04-02
2017-04-02
2018-04-03
2018-04-03
2019-04-04
2019-04-04
2020-04-04
2020-04-04
2021-04-05


In [44]:
params=['o3', 'no2', 'pm25', 'pm10']
s_date = date(2016, 4, 1)
e_date = date(2021, 4, 5)
date_list = [str(s_date + timedelta(days = x)) for x in range((e_date - s_date).days + 1)]
data_dict={date_list[i]:{param:[] for param in params} for i in range(len(date_list))}


In [35]:
from datetime import datetime
for data in responses:
    for info_by_date in data['results']:
        date_obj=datetime.fromisoformat(info_by_date['date']['local'])
        info_date=date_obj.strftime('%Y-%m-%d')
        param=info_by_date['parameter']
        if param in params:
            data_dict[info_date][param] += [info_by_date['value']]

In [36]:
def avg_list(l):
    sum = 0
    for ele in l:
        sum += ele
    return int(sum/len(l)) if len(l)>0 else  "NA"
data_dict_avg={date_list[i]:{param:avg_list(data_dict[date_list[i]][param]) for param in params} for i in range(len(date_list))}

In [37]:
df=pd.DataFrame()

In [38]:
units='(µg/m³)'
headers=['date'] + params

In [39]:
header_list={header:[] for header in headers}
for date in data_dict_avg:
    header_list['date']+= [date]
    for param in data_dict_avg[date]:
        header_list[param] += [data_dict_avg[date][param]]
for header in headers:
    if header != 'date':
        df[header + units]=header_list[header]
    else:
        df[header]=header_list[header]
    

In [40]:
df

,date,o3(µg/m³),no2(µg/m³),pm25(µg/m³),pm10(µg/m³)
0,2016-04-01,37,39,13,20
1,2016-04-02,68,24,13,21
2,2016-04-03,44,30,21,26
3,2016-04-04,54,24,15,22
4,2016-04-05,61,24,7,15
...,...,...,...,...,...
1826,2021-04-01,69,14,21,35
1827,2021-04-02,78,7,9,17
1828,2021-04-03,81,8,7,15
1829,2021-04-04,74,15,9,20


In [41]:
# determining the name of the file
file_name = 'London_Kensington_AQI_upd.xlsx'
  
# saving the excel
df.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.
